Attention Recurrent-Residual U-net

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init

Recurrent Block

In [ ]:
class Recurrent_block(nn.Module):
    def __init__(self,
                 out_ch,
                 t=2,
                 num_groups=8):
        super(Recurrent_block, self).__init__()

        self.t = t
        self.out_ch = out_ch
        self.conv = nn.Sequential(
            nn.Conv3d(out_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=True),
            nn.GroupNorm(num_groups,out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        for i in range(self.t):
            if i == 0:
                x = self.conv(x)
            out = self.conv(x + x)
        return out

Recurrent-Residual Block

In [ ]:
class RRCNN_block(nn.Module):
    def __init__(self,
                 ch_in,
                 ch_out,
                 t=2):
        super(RRCNN_block,self).__init__()
        self.RCNN = nn.Sequential(
            Recurrent_block(ch_out,t=t),
            Recurrent_block(ch_out,t=t)
        )
        self.Conv_1x1 = nn.Conv3d(ch_in,ch_out,kernel_size=1,stride=1,padding=0)

    def forward(self,x):
        x = self.Conv_1x1(x)
        x1 = self.RCNN(x)
        return x+x1


Decoder Block

In [ ]:
class up_conv(nn.Module):
    def __init__(self,
                 ch_in,
                 ch_out,
                 num_groups=8):
        super(up_conv,self).__init__()
        self.up = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv3d(ch_in,ch_out,kernel_size=3,stride=1,padding=1,bias=True),
		    nn.GroupNorm(num_groups,ch_out),
			nn.ReLU(inplace=True)
        )

    def forward(self,x):
        x = self.up(x)
        return x

Attention Block

In [ ]:
class Attention_block(nn.Module):
    def __init__(self,
                 F_g,
                 F_l,
                 F_int,
                 num_groups=8):
        super(Attention_block,self).__init__()
        self.W_g = nn.Sequential(
            nn.Conv3d(F_g, F_int, kernel_size=1,stride=1,padding=0,bias=True),
            nn.GroupNorm(num_groups,F_int)
            )
        
        self.W_x = nn.Sequential(
            nn.Conv3d(F_l, F_int, kernel_size=1,stride=1,padding=0,bias=True),
            nn.GroupNorm(num_groups,F_int)
        )

        self.psi = nn.Sequential(
            nn.Conv3d(F_int, 1, kernel_size=1,stride=1,padding=0,bias=True),
            nn.GroupNorm(num_groups,1),
            nn.Sigmoid()
        )
        
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self,g,x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1+x1)
        psi = self.psi(psi)

        return x*psi

R2AU-net 

In [ ]:

class R2AttU_Net(nn.Module):
    def __init__(self,
                 in_channel,
                 n_classes,
                 use_bias=True,
                 inplanes=32,
                 num_groups=8,
                 t=2): # Change these according to later code?
        self.in_channel = in_channel
        self.n_classes = n_classes
        self.inplanes = inplanes
        self.num_groups = num_groups
        planes = [inplanes * int(pow(2, i)) for i in range(0, 5)] # int()= integer, pow()=power, a list is created: [32,64,128,256,512,1024]         
        super(R2AttU_Net,self).__init__()
        
        self.Maxpool = nn.MaxPool3d(kernel_size=2,stride=2)
        self.Upsample = nn.Upsample(scale_factor=2)

        self.RRCNN1 = RRCNN_block(in_channel,planes[1],t=t)

        self.RRCNN2 = RRCNN_block(planes[1],planes[2],t=t)
        
        self.RRCNN3 = RRCNN_block(planes[2],planes[3],t=t)
        
        self.RRCNN4 = RRCNN_block(planes[3],planes[4],t=t)
        
        self.RRCNN5 = RRCNN_block(planes[4],planes[4],t=t)
        

        self.Up5 = up_conv(planes[4],planes[4])
        self.Att5 = Attention_block(F_g= planes[4],F_l= planes[4],F_int= planes[3])
        self.Up_RRCNN5 = RRCNN_block(planes[4], planes[4],t=t)
        
        self.Up4 = up_conv(planes[4],planes[3])
        self.Att4 = Attention_block(F_g= planes[3],F_l= planes[3],F_int= planes[2])
        self.Up_RRCNN4 = RRCNN_block(planes[4], planes[3],t=t)
        
        self.Up3 = up_conv(planes[3],planes[2])
        self.Att3 = Attention_block(F_g= planes[2],F_l= planes[2],F_int= planes[1])
        self.Up_RRCNN3 = RRCNN_block(planes[3], planes[2],t=t)
        
        self.Up2 = up_conv(planes[2],planes[1])
        self.Att2 = Attention_block(F_g= planes[1],F_l= planes[1],F_int=planes[0])
        self.Up_RRCNN2 = RRCNN_block(planes[2], planes[1],t=t)

        self.Conv_1x1 = nn.Conv3d(planes[1],output_ch,kernel_size=1,stride=1,padding=0)

        # Initializing weights in encoder and decoder according to type of layer:
        for m in self.modules():
            if isinstance(m, nn.Conv3d) or isinstance(m, nn.ConvTranspose3D): # Should this be up_conv or Upsample?
                nn.init.kaiming_normal_(m.weight,
                                        mode='fan_out',
                                        nonlinearity='relu')
            elif isinstance(m, nn.GroupNorm):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)


    def forward(self,x):
        # encoding path
        x1 = self.RRCNN1(x)

        x2 = self.Maxpool(x1)
        x2 = self.RRCNN2(x2)
        
        x3 = self.Maxpool(x2)
        x3 = self.RRCNN3(x3)

        x4 = self.Maxpool(x3)
        x4 = self.RRCNN4(x4)

        x5 = self.Maxpool(x4)
        x5 = self.RRCNN5(x5)

        # decoding + concat path
        d5 = self.Up5(x5)
        x4 = self.Att5(g=d5,x=x4)
        d5 = torch.cat((x4,d5),dim=1)
        d5 = self.Up_RRCNN5(d5)
        
        d4 = self.Up4(d5)
        x3 = self.Att4(g=d4,x=x3)
        d4 = torch.cat((x3,d4),dim=1)
        d4 = self.Up_RRCNN4(d4)

        d3 = self.Up3(d4)
        x2 = self.Att3(g=d3,x=x2)
        d3 = torch.cat((x2,d3),dim=1)
        d3 = self.Up_RRCNN3(d3)

        d2 = self.Up2(d3)
        x1 = self.Att2(g=d2,x=x1)
        d2 = torch.cat((x1,d2),dim=1)
        d2 = self.Up_RRCNN2(d2)

        d1 = self.Conv_1x1(d2)

        return d1